<h1 align="center">MNIST Dataset Notebook</h1>

<br>
<br>

### Overview

Script in this notebook is gives a demo on how tho parse the mnist dataset to csv/python memory, So that it can utilised for analysis. The below function converts the unzipped data obtained from http://yann.lecun.com/exdb/mnist/ to csv formatt and stores in the local disk.

#### Instruction to run this script:

<li>Download the datasets (total of 4 datasets) from: http://yann.lecun.com/exdb/mnist/</li>
<li>Unzipp the files and copy to the new folder called "mnist_data"</li>
<li>This newly created folder should be placed inside the folder where script is located.</li>
<li>Create an isntace of the class "DataConverterToCsv" with "[folder_name/]" as parameter </li>
<li>Run the main funtion of the instance "DataConverterToCsv".</li>

#### Load all necessary libraries:

In [82]:
import numpy as np
import os
import struct
import pandas as pd

#### Object to convert mnist data to csv format:

In [114]:
import struct
import numpy as np
import gzip
import sys
import pandas as pd


class DataConverterToCsv:
    
    """ Data converter class to conver mnist data files to csv or to load to the memory"""

    def __init__(self, data_folder_path):

        self.data_folder_path = data_folder_path
        self.train_data = None
        self.test_data = None
        self.train_label = None
        self.test_label = None
        self.train_label_data = None
        self.test_label_data = None

    def _train_test_data_parse(self, filename, indicator=None):

        try:

            try:

                processed_data = []
                
                # Data parser for type data_files converst image to numy array
                with open(self.data_folder_path+filename, 'rb') as f:  # folder path should be of format <folder_name>/
                    magic, size = struct.unpack(">II", f.read(8))
                    n_rows, n_cols = struct.unpack(">II", f.read(8))
                    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
                    data = data.reshape((size, n_rows, n_cols))

            except FileNotFoundError:

                print("Please make sure the path of the folder and ensure filename to be "
                      "default filename - {}".format(filename))
                sys.exit(1)

            except Exception as e:
                print("Something went wrong - ")
                print(e)
                sys.exit(1)

            for each_data_row in data:

                processed_data.append(each_data_row.flatten())

            processed_data = np.asarray(processed_data)
            
            # Assigning the numpy array to appropriate instance variables i.e for test and train data_file
            if indicator == "train":

                self.train_data = processed_data

                return True

            elif indicator == "test":

                self.test_data = processed_data

                return True

            else:

                return False

        except Exception as e:
            print("Something went wrong - ")
            print(e)
            sys.exit(1)

    def _train_test_label(self, filename, indicator):

        try:

            try:
                
                # # Data parser for type labels convert image labels to numy array
                with open(self.data_folder_path + filename, 'rb') as f: # folder path should be of format <folder_name>/
                    n_rows, n_cols = struct.unpack(">II", f.read(8))
                    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
                    data = data.reshape((n_cols, 1))

            except FileNotFoundError:

                print("Please make sure the path of the folder and ensure filename to be "
                      "default filename - {}".format(filename))
                sys.exit(1)

            except Exception as e:
                print("Error: Unknown please see below - ")
                print(e)
                sys.exit(1)
            
            # Assigning the numpy array to appropriate instance variables i.e for test and train labels
            if indicator == "train":

                self.train_label = data

                return True

            elif indicator == "test":

                self.test_label = data

                return True

            else:

                return False

        except Exception as e:
            print("Something went wrong - ")
            print(e)
            sys.exit(1)

    def main(self):
        
        # This is list of all mnist dataset names. Please note: it is import to define the type of dataset.
        #i.e. data_files/labels and test/train.
        file_names = {"data_files": {'train': "train-images-idx3-ubyte", 'test': "t10k-images-idx3-ubyte"},
                      "labels": {'train': "train-labels-idx1-ubyte", 'test': "t10k-labels-idx1-ubyte"}}

        try:
            
            # Calling a above appropriate funtions based on file type data_files/labels
            for file in file_names:

                if file == "data_files":
                    #  if the key: data_files call function _train_test_data_parse
                    for x in file_names[file]:
                        self._train_test_data_parse(file_names[file][x], x)

                elif file == "labels":
                    
                    #  if the key: labels call function _train_test_label
                    for x in file_names[file]:
                        self._train_test_label(file_names[file][x], x)

                else:
                    raise FileNotFoundError

        except FileNotFoundError:

            print("Please make sure files are unzipped from .gz format and file names as default.")
            sys.exit(1)

        col_names = []

        try:

            for x in range(self.train_data.shape[1]):
                col_names.append("pixel_"+str(x+1))
            
            # Convering numpy array to pandas
            train_data = pd.DataFrame(self.train_data, columns=col_names)
            test_data = pd.DataFrame(self.test_data, columns=col_names)
            train_data['labels'] = list(self.train_label)
            test_data['lables'] = list(self.test_label)
            
            # Loading pandas dataframe to csv
            train_data.to_csv(self.data_folder_path+"mnist_train_data.csv", index=False)
            test_data.to_csv(self.data_folder_path + "mnist_test_data.csv", index=False)
            self.train_label_data = train_data
            self.test_label_data = test_data

        except Exception as e:
            print("Error: Unknown please see below - ")
            print(e)
            sys.exit(1)


In [115]:
if __name__ == "__main__":
    
    inst = DataConverterToCsv("mnist_data/")
    inst.main()